In [ ]:
! pip install pg8000
! pip install psycopg2
! pip install Faker

In [2]:
import sys
import time 

import pg8000.native
import psycopg2
from faker import Faker

In [130]:
fake_customers = [] 
num = 1000000
fake = Faker()

def create_fake_customers(number: int) -> list((str, str, str, str, str, str)):
    res = []
    for k in range(number):
        if k % 50000 == 0:
            print(k / number * 100, "%", sep='')
        res.append((str(k), fake.name(), fake.address(), fake.text(), fake.date(), fake.ascii_email()))
    print("100.0%")
    return res


fake_customers = create_fake_customers(num)
print("all generated")
# to show results of using BRIN index, we need to somehow sort this by some value, for example, birthday:
fake_customers.sort(key=lambda x: x[4])
print("all sorted on 4th value")

0.0%
5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.00000000000001%
60.0%
65.0%
70.0%
75.0%
80.0%
85.0%
90.0%
95.0%
100.0%
all generated
all sorted on 4th value


In [3]:
try:
    con = psycopg2.connect(database="postgres", user="postgres", password="1", host="127.0.0.1", port="5432")
    cur = con.cursor()
except BaseException as be:
    print("Impossible to connect to db:", be)
    sys.exit(1)

In [54]:
con.commit()

In [131]:
# initialisation
cur.execute("drop table if exists customers;")
cur.execute("create table if not exists customers (id integer primary key, name text, address text, review text,birthday date, email text)")


for i in range(num):
    if i % 10000 == 0:
        print(i / num * 100, "%", sep='')
    fake_customer = fake_customers[i]
    cur.execute("INSERT INTO customers (id,name, address,review, birthday,email) VALUES ('" + fake_customer[0] + "','" + fake_customer[1] + "','" + fake_customer[2] + "','" + fake_customer[3] + "','" + fake_customer[4] + "','" + fake_customer[5] + "');")
print("100.0%")
con.commit()

0.0%
1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.000000000000001%
8.0%
9.0%
10.0%
11.0%
12.0%
13.0%
14.000000000000002%
15.0%
16.0%
17.0%
18.0%
19.0%
20.0%
21.0%
22.0%
23.0%
24.0%
25.0%
26.0%
27.0%
28.000000000000004%
28.999999999999996%
30.0%
31.0%
32.0%
33.0%
34.0%
35.0%
36.0%
37.0%
38.0%
39.0%
40.0%
41.0%
42.0%
43.0%
44.0%
45.0%
46.0%
47.0%
48.0%
49.0%
50.0%
51.0%
52.0%
53.0%
54.0%
55.00000000000001%
56.00000000000001%
56.99999999999999%
57.99999999999999%
59.0%
60.0%
61.0%
62.0%
63.0%
64.0%
65.0%
66.0%
67.0%
68.0%
69.0%
70.0%
71.0%
72.0%
73.0%
74.0%
75.0%
76.0%
77.0%
78.0%
79.0%
80.0%
81.0%
82.0%
83.0%
84.0%
85.0%
86.0%
87.0%
88.0%
89.0%
90.0%
91.0%
92.0%
93.0%
94.0%
95.0%
96.0%
97.0%
98.0%
99.0%
100.0%


In [4]:
cur.execute("select * from customers limit 10;")
print(cur.fetchall())

In [132]:
# creating indexes

cur.execute("create index a on customers using gin(to_tsvector('english', address));")
cur.execute("create index b on customers using gist(to_tsvector('english', review));")
cur.execute("create index c on customers using btree(name);")
cur.execute("create index d on customers using brin(birthday);")
cur.execute("create index e on customers using hash(email);")

In [117]:
con.commit()

In [137]:
# using indexes

start = time.time()
query = "select * from customers where to_tsvector('english', review) @@ to_tsquery('english', 'great | product') offset 10 limit 1"
cur.execute(query)
finish = time.time()
print(f"Elapsed time for the query {query}: {finish - start} seconds\nOne of results:")
for r in cur:
    print(r)


query = "explain analyse " + query
cur.execute(query)
print("\nExplain and analyse:")
for r in cur:
    print(r)

Elapsed time for the query select * from customers where to_tsvector('english', review) @@ to_tsquery('english', 'great | product') offset 10 limit 1: 0.001999378204345703 seconds
One of results:
(500885, 'Sean Daniel', '75248 Michael Square Suite 487\nLake Chelsea, RI 36523', 'Shoulder television suggest surface cover product. Research rock six suffer level.\nWrite shoulder win minute. Head these less report. Hope whatever music.', datetime.date(1975, 1, 12), 'chavezkatie@yahoo.com')

Explain and analyse:
('Limit  (cost=21.63..23.75 rows=1 width=237) (actual time=0.496..0.496 rows=1 loops=1)',)
('  ->  Index Scan using b on customers  (cost=0.41..137671.90 rows=64885 width=237) (actual time=0.190..0.494 rows=11 loops=1)',)
("        Index Cond: (to_tsvector('english'::regconfig, review) @@ '''great'' | ''product'''::tsquery)",)
('Planning Time: 0.186 ms',)
('Execution Time: 0.513 ms',)


In [123]:
start = time.time()
query = "select * from customers where to_tsvector('english', address) @@ to_tsquery('english', 'Washingtone') offset 10 limit 1"
cur.execute(query)
finish = time.time()
print(f"Elapsed time for the query {query}: {finish - start} seconds\nOne of results:")
for r in cur:
    print(r)


query = "explain analyse " + query
cur.execute(query)
print("\nExplain and analyse:")
for r in cur:
    print(r)

Elapsed time for the query select * from customers where to_tsvector('english', address) @@ to_tsquery('english', 'Washingtone') offset 10 limit 1: 0.0009922981262207031 seconds
One of results:
(16715, 'Cheryl Walker', '960 Washington Vista\nWest Jasonberg, DC 93590', 'Condition with hundred nature inside. Them take system camera employee positive. State all popular toward.', datetime.date(1982, 4, 5), 'stevensmith@smith-riley.com')

Explain and analyse:
('Limit  (cost=88.27..91.22 rows=1 width=237) (actual time=0.172..0.172 rows=1 loops=1)',)
('  ->  Bitmap Heap Scan on customers  (cost=58.75..14820.11 rows=5000 width=237) (actual time=0.162..0.171 rows=11 loops=1)',)
("        Recheck Cond: (to_tsvector('english'::regconfig, address) @@ '''washington'''::tsquery)",)
('        Heap Blocks: exact=11',)
('        ->  Bitmap Index Scan on a  (cost=0.00..57.50 rows=5000 width=0) (actual time=0.100..0.100 rows=718 loops=1)',)
("              Index Cond: (to_tsvector('english'::regconfig, a

In [122]:
start = time.time()
query = "select * from customers where name = 'David Gomez' offset 10 limit 4"
cur.execute(query)
finish = time.time()
print(f"Elapsed time for the query {query}: {finish - start} seconds\nSome of results:")
for r in cur:
    print(r)


query = "explain analyse " + query
cur.execute(query)
print("\nExplain and analyse:")
for r in cur:
    print(r)

Elapsed time for the query select * from customers where name = 'David Gomez' offset 10 limit 4: 0.0009989738464355469 seconds
Some of results:
(328242, 'David Gomez', '6335 Moore Rue Suite 834\nSouth Johnmouth, KS 81525', 'Break information gun store success. Organization in guess rest professional cause. Level big lawyer collection main.', datetime.date(1984, 7, 9), 'dherman@greene-sanders.com')
(355109, 'David Gomez', '3487 Ashley Crossroad Apt. 545\nPort Mollyland, AR 07505', 'Well attorney natural fund. Interest hospital will style their baby cut.\nNeed before two plant over though consumer. Open quickly leave should force.', datetime.date(2009, 8, 7), 'wendy99@mitchell.com')
(364203, 'David Gomez', '52609 Bailey Rest\nNew Wendy, VA 44642', 'After five seat value stock notice. Ground attack study whose ever. Reach outside out rate these us whom.\nReduce give across respond cell. Such charge name personal coach. Page probably then prove.', datetime.date(2005, 10, 5), 'englishfranci

In [134]:
start = time.time()
query = "select * from customers where birthday between '2002-01-25' and '2002-01-27' offset 10 limit 5"
cur.execute(query)
finish = time.time()
print(f"Elapsed time for the query {query}: {finish - start} seconds\nOne of results:")
for r in cur:
    print(r)


query = "explain analyse " + query
cur.execute(query)
print("\nExplain and analyse:")
for r in cur:
    print(r)

Elapsed time for the query select * from customers where birthday between '2002-01-25' and '2002-01-27' offset 10 limit 5: 0.008993148803710938 seconds
One of results:
(143659, 'Dawn Wilson', '3549 Price Spur Apt. 050\nSouth Donna, ND 11336', 'Bar gun throughout claim wind color. Page partner then newspaper key office.\nRight away color address. Early strong paper western. Eight try language short meet tree.\nWalk east build feel such girl.', datetime.date(2002, 1, 25), 'linda96@smith-mann.com')
(154078, 'Brandon Gomez', '0951 Gallagher Cove\nHernandezburgh, AK 99360', 'Management reduce their life bar entire safe. Worker various power yet study short management reveal. Charge market three somebody standard cost.', datetime.date(2002, 1, 25), 'bmckay@lopez-holt.com')
(163510, 'Marc Randolph', '2254 Mcmillan Isle\nSouth Michael, MT 14532', 'Every in company player. Cup whose street energy at office social. Economic trouble number say mother sure.', datetime.date(2002, 1, 25), 'reneeroge

In [135]:
start = time.time()
query = "select * from customers where email = 'kayla90@yahoo.com'"
cur.execute(query)
finish = time.time()
print(f"Elapsed time for the query {query}: {finish - start} seconds\nOne of results:")
for r in cur:
    print(r)

query = "explain analyse " + query
cur.execute(query)
print("\nExplain and analyse:")
for r in cur:
    print(r)

Elapsed time for the query select * from customers where email = 'kayla90@yahoo.com': 0.0009920597076416016 seconds
One of results:
(721717, 'Christopher Schultz', '25221 Peterson Stream Suite 153\nWest Keith, MN 18527', 'Environmental Democrat smile. That friend conference. Increase leader yeah.', datetime.date(2003, 7, 8), 'kayla90@yahoo.com')
(902478, 'Jeff Mueller', '53649 Haney Stravenue\nDianestad, IN 20288', 'Not nothing follow once before person. Face up your necessary. Support able service.\nFly skin hundred dark finally without point part. Choose choose foreign final training outside in guy.', datetime.date(1991, 7, 13), 'kayla90@yahoo.com')
(785632, 'Lance Figueroa', 'PSC 0516, Box 6239\nAPO AA 65928', 'View clear full much generation somebody natural design. Remember under else any.\nPass majority test the necessary operation.', datetime.date(1985, 7, 28), 'kayla90@yahoo.com')

Explain and analyse:
('Index Scan using e on customers  (cost=0.00..12.04 rows=2 width=237) (actu

In [133]:
cur.execute("analyse customers;")
cur.execute("select attname, correlation from pg_stats where tablename='customers' order by correlation desc nulls last;")
for r in cur:
    print(r)

('birthday', 0.9999986)
('review', 0.01006887)
('email', 0.0065962356)
('address', -0.0015562609)
('name', -0.009394401)
('id', -0.01056078)


In [ ]:
cur.close()
con.close()